# 時期ごとの極性・共起変遷を追う

In [12]:
ID = "mimikkyu"

POPULATION_ID = "pokemon"

PERIOD_ID = "year"

#分析対象の単語ファイル
keywords_file = "Keywords/Timelapse/%s-from-%s.txt"%(ID, POPULATION_ID)

#ストップワード抜きの形態素解析・極性分析完了したデータ
without_stopwords_text_file = "Progresses/NonStopword/%s-from-%s.txt"%(ID, POPULATION_ID)

##TFIDFモデル
tfidf_model_file = "Models/Tfidf/%s-from-%s.model"%(ID, POPULATION_ID)

#保存するファイル
sentiment_timelapse_file = "Sentiment/Timelapse/%s-from-%s-by-%s.txt"%(ID, POPULATION_ID, PERIOD_ID)

#内部的に品詞を区別する区切り文字
TOKEN_DIVIDER = "<334>"

In [13]:
#共起キーワードのストップワード（ここだけ）
local_stopwords = ["br", "ミミッキュ", "ミミ"]

## 

## 与えられた時期にインデックスを割り振る
分けたい時期にあわせて書き換える

In [14]:
from datetime import datetime

In [15]:
#これを書き換える
#万が一 -1 が出てしまうとエラーが出てしまうので注意（上限はない）
def get_time_index(time: datetime):
    #2016年から一年ごとに割り振る

    years_passed = time.year - 2016
    
    if years_passed >= 0:
        return years_passed
    else:
        return 0

#保持しているデータの形式からdatetime型に変換
def convert_time(text: str):
    return datetime.strptime(text, "%Y/%m/%d|%H/%M/%S")

#最大インデックスを算出
MAX_TIME_INDEX = get_time_index(datetime.now())

## データ読み込み

In [16]:
import pandas as pd
import numpy as np

### 処理済みデータ

In [17]:
df_main = pd.read_table(without_stopwords_text_file)

df_main.head(2)

,date,link_index,number,content,disassembled_all,disassembled_target,Sentiment
0,2017/01/10|18/43/00,57557,4,いちおつまり やっとミミッキュ理想でた <\br>,*<334>一<334>おー<334>詰まり<334>漸と<334>ミミッキュ<334>理想...,一<334>ミミッキュ<334>理想<334>br,pos
1,2017/01/10|19/04/00,57557,78,ミミッキュて生き残るの？ <\br>,*<334>ミミ<334>っ<334>きゅっ<334>って<334>生き残る<334>の<3...,ミミ<334>br,neg


### 本ノート用キーワード

In [18]:
with open(keywords_file, "r", encoding="utf-8") as f:
    keywords = f.read().split()

keywords

['z', '襷', '対策', '剣舞', '鬼火', '多い', 'レート-rate', '化けの皮']

### 母集団からのTFIDFモデルをサルベージ

In [19]:
import pickle
from copy import deepcopy

def tokenize(text):
    return text.split(TOKEN_DIVIDER)

with open(tfidf_model_file, "rb") as f:
    tfidf_model = pickle.load(f)

#対象（名詞・形容詞）を見せる用
tfidf_model_showing = deepcopy(tfidf_model)
tfidf_model_showing.ngram = (1,1)
tfidf_model_showing.tokenizer = tokenize

## 分析

### 各テキストに時期インデックスを割り振る

In [31]:
indices = []

for _, row in df_main.iterrows():
    indices.append(get_time_index(convert_time(row["date"])))

df_main["time_index"] = indices

df_main.head(3)

,date,link_index,number,content,disassembled_all,disassembled_target,Sentiment,time_index
0,2017/01/10|18/43/00,57557,4,いちおつまり やっとミミッキュ理想でた <\br>,*<334>一<334>おー<334>詰まり<334>漸と<334>ミミッキュ<334>理想...,一<334>ミミッキュ<334>理想<334>br,pos,1
1,2017/01/10|19/04/00,57557,78,ミミッキュて生き残るの？ <\br>,*<334>ミミ<334>っ<334>きゅっ<334>って<334>生き残る<334>の<3...,ミミ<334>br,neg,1
2,2017/01/10|19/39/00,57557,198,テテフよりミミッキュのほうが嫌い <\br>,*<334>ティー-tea<334>テフ-teff<334>より<334>ミミッキュ<334...,ティー-tea<334>テフ-teff<334>ミミッキュ<334>方<334>嫌い<334>br,neg,1


### キーワードごとに分析

In [42]:
#キーワードごとに
for keyword in keywords:
    scores = []
    coocs = []
    counts = []

    #時期インデックスごとに
    for time_index in range(MAX_TIME_INDEX):
        #極性スコア計算
        score = 0
        cnt = 0
        texts = []
        for _, row in df_main[df_main["time_index"] == time_index].iterrows():
            if keyword in row["disassembled_all"].split(TOKEN_DIVIDER):
                cnt += 1
                if row["Sentiment"] == "pos":
                    score += 1
                else:
                    score -= 1
                texts.append(row["disassembled_target"])
        if cnt != 0:
            score /= cnt
        else:
            score = 0
        
        ##共起
        #TFIDF計算
        if cnt != 0:
            tfidf_spycy = tfidf_model_showing.transform(texts)
            tfidf_df = pd.DataFrame(tfidf_spycy.toarray(), columns=tfidf_model_showing.get_feature_names_out())
            tfidf_mean = tfidf_df.mean(axis=0)

            #単語名と結び付け
            tfidf_mean = tfidf_mean.sort_values(ascending=False)

            #ストップワードの削除
            for word in local_stopwords:
                tfidf_mean = tfidf_mean.drop(word)

            if len(tfidf_mean) > 1:
                cooc = ", ".join(tfidf_mean[1:min(6, len(tfidf_mean))].index)
            else:
                cooc = ""
        else:
            cooc = ""
        
        scores.append(score)
        coocs.append(cooc)
        counts.append(cnt)

    df_keyword = pd.DataFrame(list(zip(counts, scores, coocs)), columns=["Scores", "Counts", "CoOccure"])

    print(keyword)
    print(df_keyword)

z
   Scores    Counts                                   CoOccure
0      32 -0.187500  z 技, 技, ゴースト-ghost, ゴースト-ghost z, ポリ-poly
1      69 -0.246377      フェアリー-fairy z, z 技, フェアリー-fairy, 技, 襷
2       0  0.000000                                           
3       0  0.000000                                           
4       0  0.000000                                           
5       0  0.000000                                           
6       0  0.000000                                           
襷
   Scores    Counts                              CoOccure
0     250 -0.256000   襷 ミミッキュ, 襷 ミミ, ギャラ-guarantee, ガブ, 玉
1     334 -0.257485              襷 ミミッキュ, 襷 ミミ, z, 良い, 剣舞
2       0  0.000000                                      
3       0  0.000000                                      
4       0  0.000000                                      
5       0  0.000000                                      
6       2  0.000000  脅威, ダイ-dye, マックス-max, トリック-trick, 不明
対策
   Scores    Counts      